# Feature creation and selection

In [ ]:
######################## LOAD IN FILES #############################
import os
import pandas as pd
import sys
import importlib

# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Define data directory
brighten_dir = os.path.join(project_root, 'BRIGHTEN_data')

# Add project root to sys.path for script usage
sys.path.append(project_root)

# Import and reload (optional) custom scripts
from scripts import preprocessing as pre
from scripts import visualization as vis
from scripts import feature_selection as fs
importlib.reload(pre)
importlib.reload(vis)
importlib.reload(fs)

df_alldays_int70 = pd.read_csv(os.path.join(brighten_dir, 'df_alldays_int70.csv'))
df_alldays_int70['dt'] = pd.to_datetime(df_alldays_int70['dt'], errors='coerce')
week_df_int70 = pd.read_csv(os.path.join(brighten_dir, 'week_df_int70.csv'))

############ LOAD in PACKAGES  #############
import numpy as np
from sklearn.linear_model import LogisticRegression


## Weekly cols
phq9_cols = ['phq9_1','phq9_2','phq9_3','phq9_4','phq9_5','phq9_6','phq9_7','phq9_8','phq9_9','phq9_sum','phq9_cat']
sds_cols = ['sds_1','sds_2','sds_3','stress','support']
sleep_cols = ['sleep_1','sleep_2','sleep_3']
gic_cols = ['mood_1']

phq2_cols = ['phq2_1','phq2_2','phq2_sum']

daily_cols_v1 = ['aggregate_communication', 'call_count',
       'call_duration', 'interaction_diversity', 'missed_interactions',
       'mobility', 'mobility_radius', 'sms_count', 'sms_length',
       'unreturned_calls']


daily_cols_v2 = ['distance_walking', 'hours_active',
       'hours_high_speed_transportation', 'hours_of_sleep',
       'hours_powered_vehicle', 'hours_stationary', 'hours_stationary_nhw',
       'hours_walking', 'location_variance',
        'cloud_cover_mean','dew_point_mean',
        'humidity_mean','temp_mean','dew_point_IQR','humidity_IQR',
        'temp_IQR','cloud_cover_IQR','cloud_cover_std','dew_point_std',
        'humidity_std','temp_std','cloud_cover_median','dew_point_median',
        'humidity_median','temp_median']


daily_cols_v3 = ['callDuration_incoming',
       'callDuration_outgoing', 'textCount', 'textCount_received',
       'textCount_sent', 'textLength_received', 'textLength_sent',
       'uniqueNumbers_calls_incoming', 'uniqueNumbers_calls_missed',
       'uniqueNumbers_calls_outgoing', 'uniqueNumbers_texts',
       'uniqueNumbers_texts_received', 'uniqueNumbers_texts_sent']

passive_cols = daily_cols_v1 + daily_cols_v2 + daily_cols_v3

added_columns = ['phq9_sum_change', 'phq9_cat_change']

mobility_cols = ['mobility', 'mobility_radius']

## Create wide versions of each vars list
wide_v1_cols = []
for var in daily_cols_v1:
    wide_v1_cols.append(f'{var}_avg_w1')
    wide_v1_cols.append(f'{var}_avg_w2')
    wide_v1_cols.append(f'{var}_slope_w2')
    wide_v1_cols.append(f'{var}_avg_w4')
    wide_v1_cols.append(f'{var}_slope_w4')

wide_v2_cols = []
for var in daily_cols_v2:
    wide_v2_cols.append(f'{var}_avg_w1')
    wide_v2_cols.append(f'{var}_avg_w2')
    wide_v2_cols.append(f'{var}_slope_w2')
    wide_v2_cols.append(f'{var}_avg_w4')
    wide_v2_cols.append(f'{var}_slope_w4')


/var/folders/fh/rstwtx7j73b0zwzkyzktnnrh0000gn/T/ipykernel_54572/3972348359.py:6: DtypeWarning: Columns (14,22,47,105,106,107,117,118,119,120,121,122,123,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  df_alldays_int70 = pd.read_csv(os.path.join(brighten_dir, 'df_alldays_int70.csv'))
/var/folders/fh/rstwtx7j73b0zwzkyzktnnrh0000gn/T/ipykernel_54572/3972348359.py:7: DtypeWarning: Columns (23,106,107,108) have mixed types. Specify dtype option on import or set low_memory=False.
  week_df_int70 = pd.read_csv(os.path.join(brighten_dir, 'week_df_int70.csv'))


In [ ]:
df_names = ['v1_day', 'v2_day', 'v1_week_phq9', 'v2_week_phq9']
id_columns = ['dt','week','num_id','participant_id']


wide_dfs_clusters = {}
for name in df_names:
    wide_dfs_clusters[name] = pd.read_csv(os.path.join(brighten_dir, f'wide_{name}_clusters.csv'))
    




In [ ]:
# PHQ9 Cat, phq9 cat change

y_col = 'pc_depression_avg_w4'
X_cols = [col for col in wide_dfs_clusters['v1_day'] if ('pc_' in col and 'w4' in col and 'depression' not in col)]
print(X_cols)
rlr_df = wide_dfs_clusters['v1_day'][['num_id', y_col] + X_cols]
print(f'df is shape {rlr_df.shape}')

X_scaled = rlr_df[X_cols].copy()
y = rlr_df[y_col].copy()

# Apply Randomized Logistic Regression
selected_features, feature_importances = fs.randomized_logistic_regression(X_scaled, y)

# Print selected feature indices
print(f"Selected Features: {selected_features}")
print(f"Feature Importances: {feature_importances}")


['pc_mobility_avg_w4', 'pc_missed_communications_avg_w4', 'pc_communications_avg_w4']
df is shape (83, 5)


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [ ]:
# PHQ9 Cat, phq9 cat change

y_cols = 'phq9_cat_bin_w6'
X_cols = wide_v1_cols
rlr_df = wide_dfs_clusters['v1_day'][['num_id', y_cols] + X_cols]
rlr_df = rlr_df.dropna()
print(f'df is shape {rlr_df.shape}')

X_scaled = rlr_df[X_cols].copy()
y = rlr_df[y_cols].copy()

# Apply Randomized Logistic Regression
selected_features, feature_importances = fs.randomized_logistic_regression(X_scaled, y)

# Print selected feature indices
print(f"Selected Features: {selected_features}")
print(f"Feature Importances: {feature_importances}")


df is shape (27, 52)


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0.0